# Ridge Regression

This notebook compares a CPU implementation and a GPU implementation of Ridge Regression.  It includes code example for doing Ridge egression using RAPIDS cuDF and cuML.

## Notebook Credits
### Authorship
Original Author: Unknown  
Last Edit: Taurean Dyer, 2/20/2019  

### Test System Specs
Test System Hardware: DGX-2  
Test System Software: Ubuntu 16.04  
RAPIDS Version: 0.5.1 - Docker Install  
Driver: 410.79  
CUDA: 10.0  

### Known Working Systems
RAPIDS Versions: 0.4, 0.5, 0.5.1  


## Let's Begin: Linear Regression
### Imports
Let's start with our Imports

In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model as sklGLM
from cuml import LinearRegression as cumlOLS
from cuml import Ridge as cumlRidge
import cudf
import os

### Helper Functions

In [2]:
from timeit import default_timer

class Timer(object):
    def __init__(self):
        self._timer = default_timer
    
    def __enter__(self):
        self.start()
        return self

    def __exit__(self, *args):
        self.stop()

    def start(self):
        """Start the timer."""
        self.start = self._timer()

    def stop(self):
        """Stop the timer. Calculate the interval in seconds."""
        self.end = self._timer()
        self.interval = self.end - self.start

In [3]:
import gzip
def load_data(nrows, ncols, cached = 'data/mortgage.npy.gz'):
    if os.path.exists(cached):
        print('use mortgage data')
        with gzip.open(cached) as f:
            X = np.load(f)
        # the 4th column is 'adj_remaining_months_to_maturity'
        # used as the label
        X = X[:,[i for i in range(X.shape[1]) if i!=4]]
        y = X[:,4:5]
        rindices = np.random.randint(0,X.shape[0]-1,nrows)
        X = X[rindices,:ncols]
        y = y[rindices]
    else:
        print('use random data')
        X = np.random.rand(nrows,ncols)
        
    df_X = pd.DataFrame({'fea%d'%i:X[:,i] for i in range(X.shape[1])})
    df_y = pd.DataFrame({'fea%d'%i:y[:,i] for i in range(y.shape[1])})
    
    return df_X, df_y

In [4]:
from sklearn.metrics import mean_squared_error
def array_equal(a,b,threshold=2e-3,with_sign=True):
    a = to_nparray(a).ravel()
    b = to_nparray(b).ravel()
    if with_sign == False:
        a,b = np.abs(a),np.abs(b)
    error = mean_squared_error(a,b)
    res = error<threshold
    return res

def to_nparray(x):
    if isinstance(x,np.ndarray) or isinstance(x,pd.DataFrame):
        return np.array(x)
    elif isinstance(x,np.float64):
        return np.array([x])
    elif isinstance(x,cudf.DataFrame) or isinstance(x,cudf.Series):
        return x.to_pandas().values
    return x    

Now that we have our Helper functions, lets start to compare the speed and results for SciKit Learn's CPU impletmenation versus RAPIDS cuML GPU impletementation. 

In [5]:
%%time
nrows = int((2**20) * 1.2)
ncols = 399

X, y = load_data(nrows,ncols)
print('data',X.shape)
print('label',y.shape)

use mortgage data
data (1258291, 399)
label (1258291, 1)
CPU times: user 14.3 s, sys: 1.08 s, total: 15.4 s
Wall time: 13.3 s


Even though the ridge regression interface of cuML is very similar to Scikit-Learn's implemetation, cuML doesn't use some of the parameters such as "copy". Also, cuML includes two different implementation of ridge regression using SVD and Eigen decomposition. Eigen decomposition based implementation is very fast but causes very small errors in the coefficients which is negligible for most of the applications. SVD is stable but slower than eigen decomposition based implementation. 

In [6]:
fit_intercept = True
normalize = False
alpha = np.array([1.0])
# eig: eigen decomposition based method, 
# svd: singular value decomposition based method,
# cd: coordinate descend.
solver = "eig" 

### Get MSE for SciKit Learn

In [7]:
%%time
reg_sk = sklGLM.Ridge(alpha=alpha, fit_intercept=fit_intercept, normalize=normalize, solver='auto')
result_sk = reg_sk.fit(X, y)

CPU times: user 15.1 s, sys: 968 ms, total: 16 s
Wall time: 4.43 s


In [8]:
%%time
y_sk = reg_sk.predict(X)
error_sk = mean_squared_error(y,y_sk)

CPU times: user 1.48 s, sys: 340 ms, total: 1.82 s
Wall time: 991 ms


### Get MSE for cuML

In [9]:
%%time
X_cudf = cudf.DataFrame.from_pandas(X)
y_cudf = y.values
y_cudf = y_cudf[:,0]
y_cudf = cudf.Series(y_cudf)

CPU times: user 2.3 s, sys: 816 ms, total: 3.12 s
Wall time: 2.13 s


In [10]:
%%time
reg_cuml = cumlRidge(alpha=alpha, fit_intercept=fit_intercept, normalize=normalize, solver=solver)
result_cuml = reg_cuml.fit(X_cudf, y_cudf)

CPU times: user 788 ms, sys: 212 ms, total: 1e+03 ms
Wall time: 995 ms


In [11]:
%%time
y_cuml = reg_cuml.predict(X_cudf)
y_cuml = to_nparray(y_cuml).ravel()
error_cuml = mean_squared_error(y,y_cuml)

CPU times: user 276 ms, sys: 20 ms, total: 296 ms
Wall time: 285 ms


## Final Comparison Between SKL and cuML
Your final output should have both MSE results close to 0 (about 1.0e-7 to 1.0e-14).  However, despite having similar answers, you should see a **massive reduction to the sys time** when using **RAPIDS cuML** versus **SciKit Learn**.  Go RAPIDS!

In [12]:
print("SKL MSE(y):")
print(error_sk)
print("CUML MSE(y):")
print(error_cuml)

SKL MSE(y):
7.84409492757235e-12
CUML MSE(y):
1.3055482e-10
